In [ ]:
# default_exp losses

# losses

> API details.

In [ ]:
#export
from torch import nn
import torch
from fastai.losses import MSELossFlat
from fastrenewables.utils import unflatten_to_ts, flatten_ts

In [ ]:
#export
class VILoss(nn.Module):
    """
    Calculate the Kullback-Leibler divergence loss.
    Parameters
    ----------
    model : dies.embedding
        the given embedding to base the loss on
    lambd : float
        scalar for the loss
    """

    def __init__(
        self,
        model,
        base_loss=torch.nn.MSELoss(),
        kl_weight=0.1,
        scale_log_likelihood=True
    ):
        super(VILoss, self).__init__()
        self.base_loss = base_loss
        self.model = model
        self.kl_weight = kl_weight
        self.scale_log_likelihood=scale_log_likelihood

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        base_loss = self.base_loss(y, y_hat)

        n_samples = max(len(y), 1)

        if self.scale_log_likelihood:
            base_loss = base_loss * n_samples

        kl = self.model.nn_kl_divergence()

        loss = base_loss + self.kl_weight * kl

        return loss

    def __repr__(self):
        s = "VILoss(\n  (base_loss):" + str(self.base_loss) + f"\n  (kl_weight): {self.kl_weight} \n)"
        return s

In [ ]:
#export
class Quantile_Score(torch.nn.Module):
    
    def reshape_1(self, x):
        return x.view(x.size()[0],1)
    
    def __init__(self, taus=[0.25, 0.5, 0.75]):
        super(Quantile_Score, self).__init__()
        
        self.taus =  torch.autograd.Variable(torch.tensor(taus, dtype=torch.float32), 
                                             requires_grad=False)
        
        self.taus = self.reshape_1(self.taus).t()
        
    def forward(self, y_hat, y):
        """
        Example:
            y = np.array([0.2, 0.1, 0.3, 0.4])
            tau=np.array([0.25,0.5,0.75])
            for each sample we have one row 
            y_hat = np.array([[0, 0.2, 0.3], 
                          [0.05, 0.1, 0.35], 
                          [0.2, 0.3, 0.6],
                          [0.3, 0.4, 0.45],])
            res = array([0.125 , 0.2   , 0.25  , 0.0625])
        """
        y = flatten_ts(y)
        y_hat = flatten_ts(y_hat)
        y = self.reshape_1(y)
        v = y - y_hat
        
        r = (torch.abs(v*(v>0).float()) * self.taus + \
             torch.abs(v*(v<0).float()) * (1-self.taus))

        # this would calculate the loss for each sample
        # r =  torch.sum(r,dim=1)
        r =  torch.sum(r)
        
        return r


In [ ]:
loss = Quantile_Score(taus=[0.25, 0.5, 0.75]).to("cpu")
probabilistic_forecasts = torch.tensor([[1,2,3],[4,5,6],[7,8,9],])
measurements = torch.tensor([2,5,8])
loss(probabilistic_forecasts, measurements)

torch.Size([3])


tensor(1.5000)

In [ ]:
#export
class CnnMSELoss(torch.nn.MSELoss):
    def __init__(self):
        """
        Calculate the MSELoss and take the mean over all features
        """
        super(CnnMSELoss, self).__init__(None, None, "mean")

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        input : pytorch.Tensor
            any given tensor. Shape: [n, ]
        target : pytorch.Tensor
            a tensor with the same shape as 'input'

        Returns
        -------
        pytorch.Tensor
            the resulting loss
        """
        return torch.mean(torch.mean(torch.mean(torch.pow((y - yhat), 2), 2), 0))

In [ ]:
#export
class VAEReconstructionLoss(nn.Module):
    def __init__(self, model, reconstruction_cost_function=MSELossFlat()):
        """
        Calculate the sum of the Kullback–Leibler divergence loss and the loss of any given function
        Parameters
        ----------
        model : dies.autoencoder
            model of the autoencoder for which the loss is to be calculated
        """
        super(VAEReconstructionLoss, self).__init__()
        self.reconstruction_cost_function = reconstruction_cost_function
        self.model = model

    def forward(self, x_hat, x):
        """

        Parameters
        ----------
        x : pytorch.Tensor
            any given tensor. Shape: [n, ]
        x_hat : pytorch.Tensor
            a tensor with the same shape as 'x'

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        mu, logvar = self.model._mu, self.model._logvar
        
        # how well do input x and output x_hat agree?
        generation_loss = self.reconstruction_cost_function(x_hat, x)
        # KLD is Kullback–Leibler divergence -- how much does one learned
        # distribution deviate from another, in this specific case the
        # learned distribution from the unit Gaussian

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        # note the negative D_{KL} in appendix B of the paper
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        # Normalise by same number of elements as in reconstruction
        KLD /= x.shape[0] * x.shape[1]

        # BCE tries to make our reconstruction as accurate as possible
        # KLD tries to push the distributions as close as possible to unit Gaussian
        
            
        return generation_loss + KLD


In [ ]:
#export
class BTuningLoss(nn.Module):
    """
    Calculate the B-Tuning Loss based on Ranking and `Tuning Pre-trained Models: A New Paradigm of Exploiting Model Hubs`
    Parameters
    ----------
        lambd : float
        scalar for the loss
    """

    def __init__(
        self,
        base_loss=MSELossFlat(),
        lambd=0.5,
    ):
        super(BTuningLoss, self).__init__()
        self.base_loss = base_loss
        self.lambd = lambd

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor where in each sample the first model is the actual prediction, where the others are from a LinearTransferModel.

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        y_hat_model = y_hat[:,0]
        y_hat_reference_models = y_hat[:,1:]
        n_reference_models = y_hat_reference_models.shape[1]
        
        base_loss = self.base_loss(y, y_hat_model)
        
        b_tuning_loss = 0
        for idx in range(n_reference_models):
#             b_tuning_loss += self.base_loss(y_hat_reference_models[:,idx], y_hat_model)
            b_tuning_loss += (y_hat_reference_models[:,idx].ravel()- y_hat_model.ravel())**2
        b_tuning_loss /= n_reference_models
        b_tuning_loss = b_tuning_loss.mean()

        loss = base_loss + self.lambd*b_tuning_loss

        return loss

    def __repr__(self):
        s = "BTuningLoss(\n  (base_loss):" + str(self.base_loss) + f"\n  (lambd): {self.lambd} \n)"
        return s

In [ ]:
# export
def btuning_rmse(y_hat, y):
    return ((y.ravel()-y_hat[:,0].ravel())**2).mean()**0.5

In [ ]:
#export

# class GaussianNegativeLogLikelihoodLoss(nn.Module):
# #     def __init__(self):
# #         """
# #         .. math::
# #         Loss = \frac{1}{N} \sum_{i} \frac{1}{2} \exp(- s_i) || y_i - \hat{y}_i ||^2 + \frac{1}{2} s_i
# #         """
# #         super(GaussianNegativeLogLikelihoodLoss, self).__init__()


#     def forward(self, output, target):
#         """

#         Parameters
#         ----------
#         y : pytorch.Tensor
#             any given tensor. Shape: [n, ]
#         output : pytorch.Tensor
#             a tensor of mu [:,0] and sigma [:,1]. Shape: [n,2]
        

#         Returns
#         -------
#         pytorch.Tensor
#             the resulting loss
#         """
#         if isinstance(output, tuple):
#             mean, sigma = output[0], output[1]
#         else:
#             mean, sigma = output[:,0], output[:,1]
            
#         squared_error = torch.pow((target - mean), 2)

#         loss = torch.mean(0.5 * (torch.exp(-sigma) * squared_error + sigma))

#         return loss

class GaussianNegativeLogLikelihoodLoss(torch.nn.Module):
    # Heteroscedastic Aleatoric Uncertainty Loss

    r"""
    .. math::
        Loss = \frac{1}{N} \sum_{i} \frac{1}{2} \exp(- s_i) || y_i - \hat{y}_i ||^2 + \frac{1}{2} s_i
    """

    def __init__(self):
        super().__init__()

    def forward(self, output, target):
        
        # flatten so that it also works for timeseries
        squared_error = torch.pow((target[:, 0].flatten() - output[:, 0].flatten()), 2)
        loss = torch.mean(0.5 * (torch.exp(-output[:, 1].flatten()) * squared_error + output[:, 1].flatten()))

        return loss



In [ ]:
#export
class RSSLoss(nn.Module):
    def __init__(self):
        """
        Calculate the Residual sum of squares loss
        """
        super(RSSLoss, self).__init__()

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting loss
        """
        return ((y - y_hat) ** 2).sum()

In [ ]:
#export

class L2SPLoss(nn.Module):
    """
    Calculate the L2SP Loss based on Explicit inductive bias for transfer learning with convolutional networks
    Parameters
    ----------
        lambd : float
        scalar for the loss
    """

    def __init__(
        self,
        source_params,
        target_params,
        base_loss=MSELossFlat(),
        lambd=0.1,
    ):
        super(L2SPLoss, self).__init__()
        self.base_loss = base_loss
        self.lambd = lambd
        self.source_params = source_params
        self.target_params = target_params

    def forward(self, y_hat, y):
        """
        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor where in each sample the first model is the actual prediction, where the others are from a LinearTransferModel.
        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """

        base_loss = self.base_loss(y, y_hat)

        l2_loss = 0
        for sp, tp in zip(
            self.source_params.parameters(), self.target_params.parameters()
        ):
            # print("source", sp)
            # print("target", tp)
            l2_loss += self.base_loss(sp, tp)
        # l2_loss = 0
        loss = base_loss + self.lambd * l2_loss

        return loss

    def __repr__(self):
        s = (
            "L2SPLoss(\n  (base_loss):"
            + str(self.base_loss)
            + f"\n  (lambd): {self.lambd} \n)"
        )
        return s

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00a_utils.ipynb.
Converted 00b_losses.ipynb.
Converted 00c_utils_blitz.ipynb.
Converted 00d_baselines.ipynb.
Converted 00e_metrics.ipynb.
Converted 00f_utils_pytorch.ipynb.
Converted 01_tabular.core.ipynb.
Converted 02_tabular.data.ipynb.
Converted 03_tabular.model.ipynb.
Converted 04_tabular.learner.ipynb.
Converted 05_timeseries.core.ipynb.
Converted 06_timeseries.data.ipynb.
Converted 07_timeseries.model.ipynb.
Converted 08_timeseries.learner.ipynb.
Converted 09_gan.core.ipynb.
Converted 10_gan.model.ipynb.
Converted 11_gan.learner.ipynb.
Converted 12_autoencoder_models.ipynb.
Converted 13_probabilistic_models.ipynb.
Converted 14_transfer_models.ipynb.
Converted 15_ensemble_models.ipynb.
Converted index.ipynb.
